In [ ]:
import os
import pickle
import random
import shutil
import sys
import time
import uuid
from collections import Counter
from glob import glob
from datetime import timedelta, datetime
from itertools import combinations

import igraph as ig
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, recall_score
from scipy import stats

In [ ]:
data_agg.loc[:, "amount"] = np.ceil(data_agg.loc[:, "amount"])
data_agg = data_agg.astype({"amount": np.uint64})
data_agg = data_agg.sort_values("amount", ascending=False).reset_index(drop=True)

In [ ]:
totals_sent = data_agg.groupby("source").agg({"amount": "sum"})["amount"].to_dict()
totals_received = data_agg.groupby("target").agg({"amount": "sum"})["amount"].to_dict()

In [ ]:
def get_communities(top_n, n_hops, data_input, pov, cp, totals, to_check_in):
    if not(0 < n_hops < 11):
        raise NotImplementedError
    if top_n < 1:
        raise ValueError
    
    result = []
    
    level_1st = data_input[data_input[pov].isin(to_check_in)].groupby(pov).head(top_n).reset_index(drop=True)
    level_1st.loc[:, "amount"] = np.array(
        [
            level_1st.loc[:, pov].apply(lambda x: totals[x]).values, 
            level_1st.loc[:, "amount"].values
        ],
        dtype=np.uint64
    ).min(axis=0)
    level_1st = level_1st.sort_values("amount", ascending=False)
    level_1st = level_1st.loc[level_1st["source"] != level_1st["target"], :].reset_index(drop=True)
    level_1st_comms = level_1st.groupby(pov).agg(nodes=(cp, list), amounts=("amount", list))
    level_1st_comms = level_1st_comms.apply(lambda x: dict(zip(x["nodes"], x["amounts"])), axis=1).to_dict()
    print(f"Processed hop #1 | {level_1st.shape[0]:,} | {len(level_1st_comms):,}")
    
    result.append(level_1st_comms)

    for n_hop in range(1, n_hops):
        if n_hop == 1:
            n_minus_1 = level_1st.copy(deep=True)
        else:
            n_minus_1 = level_nth.copy(deep=True)
        level_nth = n_minus_1.set_index(cp).join(
            level_1st.set_index(pov), lsuffix="_left", how="inner"
        ).reset_index(drop=True)
        level_nth = level_nth.loc[level_nth["source"] != level_nth["target"], :].reset_index(drop=True)
        level_nth.loc[:, "amount"] = level_nth[["amount_left", "amount"]].min(axis=1)
        del level_nth["amount_left"]
        level_nth = level_nth.groupby([pov, cp]).agg(amount=("amount", "sum")).reset_index()
        level_nth.loc[:, "amount"] = np.array(
            [
                level_nth.loc[:, pov].apply(lambda x: totals[x]).values, 
                level_nth.loc[:, "amount"].values
            ],
            dtype=np.uint64
        ).min(axis=0)
        level_nth = level_nth.sort_values("amount", ascending=False).reset_index(drop=True)
        level_nth = level_nth.groupby(pov).head(top_n).reset_index(drop=True)
        level_nth_comms = level_nth.groupby(pov).agg(nodes=(cp, list), amounts=("amount", list))
        level_nth_comms = level_nth_comms.apply(lambda x: dict(zip(x["nodes"], x["amounts"])), axis=1).to_dict()
        print(f"Processed hop #{n_hop + 1} | {level_nth.shape[0]:,} | {len(level_nth_comms):,}")

        result.append(level_nth_comms)
    
    del level_1st
    del n_minus_1
    
    return result

In [ ]:
print("Processing communities_as_source\n")
communities_as_source = get_communities(50, 4, data_agg, "source", "target", totals_sent, nodes_source)

In [ ]:
print("Processing communities_as_target\n")
communities_as_target = get_communities(50, 4, data_agg, "target", "source", totals_received, nodes_target)

In [ ]:
print("Processing communities_as_passthrough\n")
communities_as_passthrough = get_communities(
    50, 4, data_agg.loc[data_agg["source"].isin(nodes_passthrough), :], "source", "target", 
    totals_received, nodes_passthrough
)

In [ ]:
communities_as_source_features = []
for node in nodes_source.intersection(communities_as_source[0].keys()):
    all_nodes = set()
    node_comm_stats = {"key": node}
    for index, communities in enumerate(communities_as_source):
        n_hop = index + 1
        node_comm_stats[f"hop_{n_hop}_number_of_accounts"] = 0
        if index:
            node_comm_stats[f"hop_{n_hop}_number_of_distinct_accounts"] = 0
            node_comm_stats[f"hop_{n_hop}_number_of_new_accounts"] = 0
        node_comm_stats[f"hop_{n_hop}_max_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_mean_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_median_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_std_transferred"] = 0
        if not communities.get(node):
            continue
        nodes_community, amounts_community = zip(*communities[node].items())
        if not index:
            sum_prev = totals_sent[node]
        else:
            sum_prev = sum(communities_as_source[index - 1][node].values())
        sum_prev = min([totals_sent[node], sum_prev])
        amounts_community_adjusted = np.array(amounts_community, dtype=np.float64)
        amounts_community_adjusted /= sum_prev
        amounts_community_adjusted[amounts_community_adjusted > 1] = 1
        sum_this = sum(communities[node].values())
        perc_transferred = (sum_this / sum_prev) if sum_prev > sum_this else 1
        amounts_community_adjusted *= perc_transferred
        number_of_new_accounts = len(set(nodes_community) - all_nodes)
        all_nodes = all_nodes.union(nodes_community)
        node_comm_stats[f"hop_{n_hop}_number_of_accounts"] = len(nodes_community)
        if index:
            node_comm_stats[f"hop_{n_hop}_number_of_distinct_accounts"] = len(all_nodes)
            node_comm_stats[f"hop_{n_hop}_number_of_new_accounts"] = number_of_new_accounts
        node_comm_stats[f"hop_{n_hop}_max_transferred"] = np.max(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_mean_transferred"] = np.mean(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_median_transferred"] = np.median(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_std_transferred"] = np.std(amounts_community_adjusted)
        # node_comm_stats[f"hop_{n_hop}_skew_transferred"] = stats.skew(amounts_community_adjusted)
        # node_comm_stats[f"hop_{n_hop}_kurtosis_transferred"] = stats.kurtosis(amounts_community_adjusted)
    communities_as_source_features.append(node_comm_stats)

communities_as_source_features = pd.DataFrame(communities_as_source_features)

In [ ]:
communities_as_target_features = []
for node in nodes_target.intersection(communities_as_target[0].keys()):
    all_nodes = set()
    node_comm_stats = {"key": node}
    for index, communities in enumerate(communities_as_target):
        n_hop = index + 1
        node_comm_stats[f"hop_{n_hop}_number_of_accounts"] = 0
        if index:
            node_comm_stats[f"hop_{n_hop}_number_of_distinct_accounts"] = 0
            node_comm_stats[f"hop_{n_hop}_number_of_new_accounts"] = 0
        node_comm_stats[f"hop_{n_hop}_max_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_mean_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_median_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_std_transferred"] = 0
        if not communities.get(node):
            continue
        nodes_community, amounts_community = zip(*communities[node].items())
        if not index:
            sum_prev = totals_received[node]
        else:
            sum_prev = sum(communities_as_target[index - 1][node].values())
        sum_prev = min([totals_received[node], sum_prev])
        amounts_community_adjusted = np.array(amounts_community, dtype=np.float64)
        amounts_community_adjusted /= sum_prev
        amounts_community_adjusted[amounts_community_adjusted > 1] = 1
        sum_this = sum(communities[node].values())
        perc_transferred = (sum_this / sum_prev) if sum_prev > sum_this else 1
        amounts_community_adjusted *= perc_transferred
        number_of_new_accounts = len(set(nodes_community) - all_nodes)
        all_nodes = all_nodes.union(nodes_community)
        node_comm_stats[f"hop_{n_hop}_number_of_accounts"] = len(nodes_community)
        if index:
            node_comm_stats[f"hop_{n_hop}_number_of_distinct_accounts"] = len(all_nodes)
            node_comm_stats[f"hop_{n_hop}_number_of_new_accounts"] = number_of_new_accounts
        node_comm_stats[f"hop_{n_hop}_max_transferred"] = np.max(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_mean_transferred"] = np.mean(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_median_transferred"] = np.median(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_std_transferred"] = np.std(amounts_community_adjusted)
        # node_comm_stats[f"hop_{n_hop}_skew_transferred"] = stats.skew(amounts_community_adjusted)
        # node_comm_stats[f"hop_{n_hop}_kurtosis_transferred"] = stats.kurtosis(amounts_community_adjusted)
    communities_as_target_features.append(node_comm_stats)

communities_as_target_features = pd.DataFrame(communities_as_target_features)

In [ ]:
communities_as_passthrough_features = []
for node in nodes_passthrough.intersection(communities_as_passthrough[0].keys()):
    all_nodes = set()
    node_comm_stats = {"key": node}
    for index, communities in enumerate(communities_as_passthrough):
        n_hop = index + 1
        node_comm_stats[f"hop_{n_hop}_number_of_accounts"] = 0
        if index:
            node_comm_stats[f"hop_{n_hop}_number_of_distinct_accounts"] = 0
            node_comm_stats[f"hop_{n_hop}_number_of_new_accounts"] = 0
        node_comm_stats[f"hop_{n_hop}_max_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_mean_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_median_transferred"] = 0
        node_comm_stats[f"hop_{n_hop}_std_transferred"] = 0
        if not communities.get(node):
            continue
        nodes_community, amounts_community = zip(*communities[node].items())
        if not index:
            sum_prev = totals_received[node]
        else:
            sum_prev = sum(communities_as_passthrough[index - 1][node].values())
        sum_prev = min([totals_received[node], sum_prev])
        amounts_community_adjusted = np.array(amounts_community, dtype=np.float64)
        amounts_community_adjusted /= sum_prev
        amounts_community_adjusted[amounts_community_adjusted > 1] = 1
        sum_this = sum(communities[node].values())
        perc_transferred = (sum_this / sum_prev) if sum_prev > sum_this else 1
        amounts_community_adjusted *= perc_transferred
        number_of_new_accounts = len(set(nodes_community) - all_nodes)
        all_nodes = all_nodes.union(nodes_community)
        node_comm_stats[f"hop_{n_hop}_number_of_accounts"] = len(nodes_community)
        if index:
            node_comm_stats[f"hop_{n_hop}_number_of_distinct_accounts"] = len(all_nodes)
            node_comm_stats[f"hop_{n_hop}_number_of_new_accounts"] = number_of_new_accounts
        node_comm_stats[f"hop_{n_hop}_max_transferred"] = np.max(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_mean_transferred"] = np.mean(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_median_transferred"] = np.median(amounts_community_adjusted)
        node_comm_stats[f"hop_{n_hop}_std_transferred"] = np.std(amounts_community_adjusted)
        # node_comm_stats[f"hop_{n_hop}_skew_transferred"] = stats.skew(amounts_community_adjusted)
        # node_comm_stats[f"hop_{n_hop}_kurtosis_transferred"] = stats.kurtosis(amounts_community_adjusted)
    communities_as_passthrough_features.append(node_comm_stats)

communities_as_passthrough_features = pd.DataFrame(communities_as_passthrough_features)

In [ ]:
communities_as_source_features.set_index("key", inplace=True)
communities_as_target_features.set_index("key", inplace=True)
communities_as_passthrough_features.set_index("key", inplace=True)

In [ ]:
new_types = {}
for k, v in communities_as_source_features.dtypes.to_dict().items():
    if str(v).startswith("int"):
        new_types[k] = np.uint8
    elif str(v).startswith("float"):
        new_types[k] = np.float16
    elif str(v).startswith("uint"):
        pass
    else:
        raise Exception(k, v)

communities_as_source_features = communities_as_source_features.astype(new_types)
communities_as_target_features = communities_as_target_features.astype(new_types)
communities_as_passthrough_features = communities_as_passthrough_features.astype(new_types)

In [ ]:
communities_as_source_features.columns = [f"{s.G_GLOB_PREFIX}{x}" for x in communities_as_source_features.columns]
communities_as_target_features.columns = [f"{s.G_GLOB_PREFIX}{x}" for x in communities_as_target_features.columns]
communities_as_passthrough_features.columns = [f"{s.G_GLOB_PREFIX}{x}" for x in communities_as_passthrough_features.columns]